In [1]:
import glob
import os
import csv
import numpy as np
import pandas as pd

In [2]:
output_path = './CIFAR10/'
exp_prefixes = [
    'adaptive_disparity_sampling', 'confidence_disparity_sampling', 'dbal',
    'ENT', 'margin', 'uncertainty_propagation_sampling', 'BALD',
    'contextual_clarity_sampling', 'decision_boundary_exploration',
    'entropy', 'random', 'vaal', 'baldD', 'coreset', 'ensemble_var_R',
    'entropy_gradient_divergence', 'uncertainty'
]
dataset_name = 'CIFAR10'
model = 'resnet18'
output_file_name = f'{dataset_name}_{model}_ALL.csv'

exp_list = []
for exp in exp_prefixes:
    # Use '*' at both ends of the pattern to match any position
    matches = glob.glob(os.path.join(output_path, model, f'*{exp}*'))
    exp_list.extend(matches)

# Display all the matched directories
print(f'Matched directories:\n{exp_list}')


Matched directories:
['./CIFAR10/resnet18/adaptive_disparity_sampling', './CIFAR10/resnet18/confidence_disparity_sampling', './CIFAR10/resnet18/dbal', './CIFAR10/resnet18/ENT_1', './CIFAR10/resnet18/margin', './CIFAR10/resnet18/uncertainty_propagation_sampling', './CIFAR10/resnet18/BALD', './CIFAR10/resnet18/contextual_clarity_sampling', './CIFAR10/resnet18/decision_boundary_exploration', './CIFAR10/resnet18/entropy_gradient_divergence', './CIFAR10/resnet18/entropy', './CIFAR10/resnet18/random', './CIFAR10/resnet18/vaal', './CIFAR10/resnet18/baldD', './CIFAR10/resnet18/coreset', './CIFAR10/resnet18/entropy_gradient_divergence', './CIFAR10/resnet18/uncertainty', './CIFAR10/resnet18/uncertainty_propagation_sampling']


In [3]:
def exp_to_al_method(exp):
    if 'adaptive_disparity_sampling' in exp:
        return 'AdaptiveDisparity'
    elif 'confidence_disparity_sampling' in exp:
        return 'ConfidenceDisparity'
    elif 'dbal' in exp:
        return 'DBAL'
    elif 'ENT' in exp or 'entropy' in exp:
        return 'MaxEntropy'
    elif 'margin' in exp:
        return 'MinMargin'
    elif 'uncertainty_propagation_sampling' in exp:
        return 'UncertaintyPropagation'
    elif 'BALD' in exp:
        return 'BALD'
    elif 'contextual_clarity_sampling' in exp:
        return 'ContextualClarity'
    elif 'decision_boundary_exploration' in exp:
        return 'DecisionBoundary'
    elif 'random' in exp:
        return 'Random'
    elif 'vaal' in exp:
        return 'VAAL'
    elif 'baldD' in exp:
        return 'BALDD'
    elif 'coreset' in exp:
        return 'Coreset'
    elif 'ensemble_var_R' in exp or 'ENS' in exp:
        return 'ENSvarR'
    elif 'entropy_gradient_divergence' in exp:
        return 'EntropyGradientDivergence'
    elif 'uncertainty' in exp:
        return 'Uncertainty'
    else:
        return 'Unknown'

# Apply the function to each element in exp_list
method_mapping = {exp: exp_to_al_method(exp) for exp in exp_list}

# Display the mapping
print("Mapping of directories to active learning methods:")
for exp, method in method_mapping.items():
    print(f"{exp} -> {method}")


Mapping of directories to active learning methods:
./CIFAR10/resnet18/adaptive_disparity_sampling -> AdaptiveDisparity
./CIFAR10/resnet18/confidence_disparity_sampling -> ConfidenceDisparity
./CIFAR10/resnet18/dbal -> DBAL
./CIFAR10/resnet18/ENT_1 -> MaxEntropy
./CIFAR10/resnet18/margin -> MinMargin
./CIFAR10/resnet18/uncertainty_propagation_sampling -> UncertaintyPropagation
./CIFAR10/resnet18/BALD -> BALD
./CIFAR10/resnet18/contextual_clarity_sampling -> ContextualClarity
./CIFAR10/resnet18/decision_boundary_exploration -> DecisionBoundary
./CIFAR10/resnet18/entropy_gradient_divergence -> MaxEntropy
./CIFAR10/resnet18/entropy -> MaxEntropy
./CIFAR10/resnet18/random -> Random
./CIFAR10/resnet18/vaal -> VAAL
./CIFAR10/resnet18/baldD -> BALDD
./CIFAR10/resnet18/coreset -> Coreset
./CIFAR10/resnet18/uncertainty -> Uncertainty


In [4]:
columns = ['AL_Method', 'Episode', 'TestAccuracy']
data = []

In [5]:
for exp in exp_list:
    file_path = os.path.join(exp, 'plot_episode_yvalues.txt')
    file = open(file_path, "r")
    values = file.readlines()
    values = [float(v[:-1]) for v in values]
    data_tuple = []
    for idx in range(len(values)):
        data_tuple.append([exp_to_al_method(exp), idx, values[idx]])
    data.extend(data_tuple)

In [6]:
df = pd.DataFrame(data, columns = columns)
df.Episode = df.Episode.apply(lambda x: int((x+1)*10))

In [7]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import warnings
warnings.filterwarnings('ignore')

from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Arial']})
rc('text', usetex=True)

plt.rcParams["axes.labelweight"] = "normal"
plt.rcParams["font.weight"] = "normal"

sns.set_style("darkgrid")

In [8]:

al_methods = ['LeastConfidence','MaxEntropy','MinMargin', 'ENSvarR','Coreset','VAAL', 'DBAL', 'Random',]
colors = ['black', 'red', 'springgreen',  'blue', 'peru', 'magenta', 'gold', 'cyan', 'darkviolet', 'orange']
colors = colors[:len(set(df.AL_Method))]

# print(index)
#     if index != 7:
#         continue
fig, ax = plt.subplots(figsize=(7,6))

sns.set(rc={"lines.linewidth": 2})
# sns.set_style("ticks")
sns.lineplot(x="Episode", y="TestAccuracy", hue="AL_Method", data=df, \
             ax=ax, palette=colors, linewidth = 3, legend=True)
ax.set_xlabel('\% of Data Labeled', size = 22, )
ax.set_ylabel('Test Accuracy', size = 22,)

ax.patch.set_edgecolor('black')  
ax.patch.set_linewidth('2')

ax.set_title(dataset_name, size = 30)

ax.legend(loc='lower left', bbox_to_anchor=(1.1, 0.1), shadow=True, markerscale=1, ncol=1, prop={'size': 20})

x_vals = np.arange(10,70,10)

xticks = ax.set_xticks(x_vals)
ax.set_xticklabels(ax.get_xticks(), size = 25)
ax.set_yticklabels(ax.get_yticks(), size = 25)  


axins2 = ax.inset_axes([0.55, 0.1, 0.4, 0.4])
sns.set_style("darkgrid")
# sub region of the original image
if dataset_name == 'CIFAR10':
    x1, x2, y1, y2 = 55, 61, 89, 92
else:
    x1, x2, y1, y2 = 55, 61, 52, 56.7
axins2.set_xlim(x1, x2)
axins2.set_ylim(y1, y2)
ax.indicate_inset_zoom(axins2, edgecolor="black")
sns.lineplot(x="Episode", y="TestAccuracy", hue="AL_Method", data=df,\
             ax=axins2, legend=False, palette=colors, linewidth=2)
axins2.set_xlabel("")
axins2.set_ylabel("")
axins2.set_xticklabels('')

axins2.set_yticklabels([str(round(float(label), 1)) for label in axins2.get_yticks()])
# axins2.set_yticklabels('')
axins2.patch.set_edgecolor('black')  
axins2.patch.set_linewidth('2')
axins2.tick_params(axis = "x", which = "both", bottom = False, top = False)

ax.indicate_inset_zoom(axins2, edgecolor="black")

plt.savefig(f'./{dataset_name}_AL.png', dpi=300, format='png', bbox_inches = "tight")
plt.show()

TypeError: must be real number, not str

Error in callback <function _draw_all_if_interactive at 0x77e4278c3a60> (for post_execute):


TypeError: must be real number, not str

TypeError: must be real number, not str

<Figure size 700x600 with 1 Axes>

In [58]:
df2 = df.groupby(['AL_Method', 'Episode']).mean()['TestAccuracy'].reset_index()
df2.columns = ['AL_Method', 'Episode', 'Mean']
df2['SD'] = df.groupby(['AL_Method', 'Episode']).std()['TestAccuracy'].reset_index().TestAccuracy

In [59]:
df2.sort_values(['Mean', 'SD'], ascending=False).loc[df.Episode==60, :].iloc[:, [0, 2, 3]]

,AL_Method,Mean,SD
17,DBAL,91.670000,0.230651
29,LeastConfidence,91.510000,0.087178
5,BALD,91.470000,0.293087
11,Coreset,91.433333,0.090738
35,MaxEntropy,91.373333,0.363639
41,MinMargin,91.333333,0.234592
23,ENSvarR,89.866667,0.127410
47,Random,89.803333,0.230290
53,VAAL,89.690000,0.115326


In [9]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Subset of active learning methods to match
al_methods = [
    'AdaptiveDisparity', 'ConfidenceDisparity', 'DBAL', 'MaxEntropy',
    'MinMargin', 'UncertaintyPropagation', 'BALD', 'ContextualClarity',
    'DecisionBoundary', 'Random', 'VAAL', 'BALDD', 'Coreset', 'ENSvarR', 
    'Uncertainty'
]

# Corresponding colors for the selected methods
colors = [
    'black', 'red', 'springgreen', 'blue', 'peru', 'magenta', 'gold', 
    'cyan', 'darkviolet', 'orange', 'pink', 'brown', 'olive', 'teal', 'purple'
]

# Adjust the colors to match the unique methods in the DataFrame
unique_methods = list(set(df['AL_Method']))
colors = colors[:len(unique_methods)]

# Plotting
fig, ax = plt.subplots(figsize=(7, 6))
sns.set(rc={"lines.linewidth": 2})

# Lineplot with the specified subset of active learning methods
sns.lineplot(
    x="Episode", y="TestAccuracy", hue="AL_Method", data=df,
    ax=ax, palette=colors, linewidth=3, legend=True
)

ax.set_xlabel('% of Data Labeled', size=22)
ax.set_ylabel('Test Accuracy', size=22)

ax.patch.set_edgecolor('black')  
ax.patch.set_linewidth('2')

ax.set_title(dataset_name, size=30)

ax.legend(
    loc='lower left', bbox_to_anchor=(1.1, 0.1), shadow=True,
    markerscale=1, ncol=1, prop={'size': 20}
)

x_vals = np.arange(10, 70, 10)
ax.set_xticks(x_vals)
ax.set_xticklabels(ax.get_xticks(), size=25)
ax.set_yticklabels(ax.get_yticks(), size=25)

# Inset zoom plot
axins2 = ax.inset_axes([0.55, 0.1, 0.4, 0.4])
sns.set_style("darkgrid")

# Sub-region of the original image
if dataset_name == 'CIFAR10':
    x1, x2, y1, y2 = 55, 61, 89, 92
else:
    x1, x2, y1, y2 = 55, 61, 52, 56.7

axins2.set_xlim(x1, x2)
axins2.set_ylim(y1, y2)
ax.indicate_inset_zoom(axins2, edgecolor="black")

sns.lineplot(
    x="Episode", y="TestAccuracy", hue="AL_Method", data=df,
    ax=axins2, legend=False, palette=colors, linewidth=2
)

axins2.set_xlabel("")
axins2.set_ylabel("")
axins2.set_xticklabels('')
axins2.set_yticklabels([str(round(float(label), 1)) for label in axins2.get_yticks()])

axins2.patch.set_edgecolor('black')  
axins2.patch.set_linewidth('2')
axins2.tick_params(axis="x", which="both", bottom=False, top=False)

plt.savefig(f'./{dataset_name}_AL.png', dpi=300, format='png', bbox_inches="tight")
plt.show()


TypeError: must be real number, not str

Error in callback <function _draw_all_if_interactive at 0x77e4278c3a60> (for post_execute):


TypeError: must be real number, not str

TypeError: must be real number, not str

<Figure size 700x600 with 1 Axes>

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Subset of active learning methods to match
al_methods = [
    'AdaptiveDisparity', 'ConfidenceDisparity', 'DBAL', 'MaxEntropy',
    'MinMargin', 'UncertaintyPropagation', 'BALD', 'ContextualClarity',
    'DecisionBoundary', 'Random', 'VAAL', 'BALDD', 'Coreset', 'ENSvarR', 
    'Uncertainty'
]

# Corresponding colors for the selected methods
colors = [
    'black', 'red', 'springgreen', 'blue', 'peru', 'magenta', 'gold', 
    'cyan', 'darkviolet', 'orange', 'pink', 'brown', 'olive', 'teal', 'purple'
]

# Adjust the colors to match the unique methods in the DataFrame
unique_methods = list(set(df['AL_Method']))
colors = colors[:len(unique_methods)]

# Plotting
fig, ax = plt.subplots(figsize=(7, 6))
sns.set(rc={"lines.linewidth": 2})

# Lineplot with the specified subset of active learning methods
sns.lineplot(
    x="Episode", y="TestAccuracy", hue="AL_Method", data=df,
    ax=ax, palette=colors, linewidth=3, legend=True
)

ax.set_xlabel('% of Data Labeled', size=22)
ax.set_ylabel('Test Accuracy', size=22)

ax.patch.set_edgecolor('black')  
ax.patch.set_linewidth('2')

ax.set_title(dataset_name, size=30)

ax.legend(
    loc='lower left', bbox_to_anchor=(1.1, 0.1), shadow=True,
    markerscale=1, ncol=1, prop={'size': 20}
)

x_vals = np.arange(10, 70, 10)
ax.set_xticks(x_vals)
ax.set_xticklabels(ax.get_xticks(), size=25)
ax.set_yticklabels(ax.get_yticks(), size=25)

# Inset zoom plot
axins2 = ax.inset_axes([0.55, 0.1, 0.4, 0.4])
sns.set_style("darkgrid")

# Sub-region of the original image
if dataset_name == 'CIFAR10':
    x1, x2, y1, y2 = 55, 61, 89, 92
else:
    x1, x2, y1, y2 = 55, 61, 52, 56.7

axins2.set_xlim(x1, x2)
axins2.set_ylim(y1, y2)
ax.indicate_inset_zoom(axins2, edgecolor="black")

sns.lineplot(
    x="Episode", y="TestAccuracy", hue="AL_Method", data=df,
    ax=axins2, legend=False, palette=colors, linewidth=2
)

axins2.set_xlabel("")
axins2.set_ylabel("")
axins2.set_xticklabels('')
axins2.set_yticklabels([str(round(float(label), 1)) for label in axins2.get_yticks()])

axins2.patch.set_edgecolor('black')  
axins2.patch.set_linewidth('2')
axins2.tick_params(axis="x", which="both", bottom=False, top=False)

plt.savefig(f'./{dataset_name}_AL.png', dpi=300, format='png', bbox_inches="tight")
plt.show()


TypeError: must be real number, not str

Error in callback <function _draw_all_if_interactive at 0x77e4278c3a60> (for post_execute):


TypeError: must be real number, not str

TypeError: must be real number, not str

<Figure size 700x600 with 1 Axes>

In [11]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Subset of active learning methods to match
al_methods = [
    'AdaptiveDisparity', 'ConfidenceDisparity', 'DBAL', 'MaxEntropy',
    'MinMargin', 'UncertaintyPropagation', 'BALD', 'ContextualClarity',
    'DecisionBoundary', 'Random', 'VAAL', 'BALDD', 'Coreset', 'ENSvarR', 
    'Uncertainty'
]

# Corresponding colors for the selected methods
colors = [
    'black', 'red', 'springgreen', 'blue', 'peru', 'magenta', 'gold', 
    'cyan', 'darkviolet', 'orange', 'pink', 'brown', 'olive', 'teal', 'purple'
]

# Adjust the colors to match the unique methods in the DataFrame
unique_methods = list(set(df['AL_Method']))
colors = colors[:len(unique_methods)]

# Plotting
fig, ax = plt.subplots(figsize=(7, 6))
sns.set(rc={"lines.linewidth": 2})

# Lineplot with the specified subset of active learning methods
sns.lineplot(
    x="Episode", y="TestAccuracy", hue="AL_Method", data=df,
    ax=ax, palette=colors, linewidth=3, legend=True
)

ax.set_xlabel('% of Data Labeled', size=22)
ax.set_ylabel('Test Accuracy', size=22)

ax.patch.set_edgecolor('black')  
ax.patch.set_linewidth('2')

ax.set_title(dataset_name, size=30)

ax.legend(
    loc='lower left', bbox_to_anchor=(1.1, 0.1), shadow=True,
    markerscale=1, ncol=1, prop={'size': 20}
)

x_vals = np.arange(10, 70, 10)
ax.set_xticks(x_vals)
ax.set_xticklabels(ax.get_xticks(), size=25)
ax.set_yticklabels(ax.get_yticks(), size=25)

# Inset zoom plot
axins2 = ax.inset_axes([0.55, 0.1, 0.4, 0.4])
sns.set_style("darkgrid")

# Sub-region of the original image
if dataset_name == 'CIFAR10':
    x1, x2, y1, y2 = 55, 61, 89, 92
else:
    x1, x2, y1, y2 = 55, 61, 52, 56.7

axins2.set_xlim(x1, x2)
axins2.set_ylim(y1, y2)
ax.indicate_inset_zoom(axins2, edgecolor="black")

sns.lineplot(
    x="Episode", y="TestAccuracy", hue="AL_Method", data=df,
    ax=axins2, legend=False, palette=colors, linewidth=2
)

axins2.set_xlabel("")
axins2.set_ylabel("")
axins2.set_xticklabels('')
axins2.set_yticklabels([str(round(float(label), 1)) for label in axins2.get_yticks()])

axins2.patch.set_edgecolor('black')  
axins2.patch.set_linewidth('2')
axins2.tick_params(axis="x", which="both", bottom=False, top=False)

plt.savefig(f'./{dataset_name}_AL.png', dpi=300, format='png', bbox_inches="tight")
plt.show()


TypeError: must be real number, not str

Error in callback <function _draw_all_if_interactive at 0x77e4278c3a60> (for post_execute):


TypeError: must be real number, not str

TypeError: must be real number, not str

<Figure size 700x600 with 1 Axes>